## 로지스틱 회귀

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/강의자료/머신러닝기초/3일차_회귀

/content/drive/MyDrive/강의자료/머신러닝기초/3일차_회귀


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression

```python
df = pd.read_csv('./datasets/초등_4학년_수학_차시문항풀이이력.csv', index_col=0)
pd.set_option('display.max_columns', len(df.columns))
df.head()
```

In [3]:
df = pd.read_csv('./datasets/초등_4학년_수학_차시문항풀이이력.csv', index_col=0)

In [4]:
pd.set_option('display.max_columns', len(df.columns))
df.head()

,사용자ID,차시코드,단원코드,단원제목,강의내용,강의명,학년,학기,시험구분,강의구분,강의타입,동영상재생시간,실제재생시간,확인문제점수,학습일,문항번호,문항코드,사용자입력,정오답,영역,대단원코드,대단원제목,중단원코드,중단원제목,소단원코드,소단원제목,토픽코드,토픽제목,난이도,평가영역
0,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,AAA,0,8,-1,2022-07-04 21:00:03,1.0,30016642.0,10000,O,MA,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233514.0,다섯 자리 수의 이해,12234054.0,모형 돈 세어 보기,2.0,91.0
1,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,AAA,0,8,-1,2022-07-04 21:00:03,2.0,30016654.0,1000/^ROW^/100/^ROW^/10/^ROW^/1,O,MA,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233514.0,다섯 자리 수의 이해,12233718.0,10000 알아보기,2.0,91.0
2,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,AAA,0,8,-1,2022-07-04 21:00:03,3.0,30016589.0,713758/^ROW^/칠만십삼천칠배오십팔,X,MA,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233514.0,다섯 자리 수의 이해,12234058.0,"10000이 ■, 1000이 ▲ ……인 다섯 자리 수를 나타내기",2.0,91.0
3,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,AAA,0,8,-1,2022-07-04 21:00:03,4.0,30016571.0,60000/^ROW^/3000/^ROW^/100/^ROW^/90/^ROW^/7,O,MA,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233515.0,다섯 자리 수의 자릿값,12233721.0,각 자릿수의 합으로 나타내기,3.0,91.0
4,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,AAA,0,8,-1,2022-07-04 21:00:03,5.0,30016608.0,천/^ROW^/2000,O,MA,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233515.0,다섯 자리 수의 자릿값,12233719.0,각 자리의 숫자와 나타내는 값 알아보기,3.0,91.0


```python
df['정오답'] =  df['정오답'].map({'O': 1, 'X': 0})
```

In [5]:
df['정오답'] =  df['정오답'].map({'O': 1, 'X': 0})

Q. 어떤 피쳐가 정오답에 영향을 미칠까?

```python
df = df[['정오답', '강의타입', '동영상재생시간', '실제재생시간', '학습일', '문항코드', '대단원코드', '중단원코드', '소단원코드', '토픽코드', '난이도', '평가영역']]
```

In [6]:
df = df[['정오답', '강의타입', '동영상재생시간', '실제재생시간', '학습일', '문항코드', '대단원코드', '중단원코드', '소단원코드', '토픽코드', '난이도', '평가영역']]

```python
df = df.dropna(subset=['정오답'])
```

In [7]:
df = df.dropna(subset=['정오답'])

```python
df['학습일'] = df.학습일.apply(pd.to_datetime)
df['hour'] = df.학습일.apply(lambda x: x.hour)
```

In [9]:
df['학습일'] = df.학습일.apply(pd.to_datetime)

In [10]:
df['hour'] = df.학습일.apply(lambda x: x.hour)

카테고리형 변수 원핫인코딩

```python
df = pd.get_dummies(df, columns=['강의타입', '문항코드', '대단원코드', '중단원코드', '소단원코드', '토픽코드', '난이도', '평가영역', 'hour'])
```

In [11]:
df = pd.get_dummies(df, columns=['강의타입', '문항코드', '대단원코드', '중단원코드', '소단원코드', '토픽코드', '난이도', '평가영역', 'hour'])

```python
df = df.drop(labels='학습일', axis=1)
```

In [12]:
df = df.drop(labels='학습일', axis=1)

```python
X = df.iloc[:,1:].to_numpy()
Y = df['정오답'].to_numpy()
```

In [13]:
X = df.iloc[:,1:].to_numpy()
Y = df['정오답'].to_numpy()

```python
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train , X_test, y_train , y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
```

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train , X_test, y_train , y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

```python
from sklearn.metrics import accuracy_score, roc_auc_score

# 로지스틱 회귀를 이용하여 학습 및 예측 수행.
# solver인자값을 생성자로 입력하지 않으면 solver='lbfgs'  
lr_clf = LogisticRegression() # solver='lbfgs'
lr_clf.fit(X_train, y_train)
lr_preds = lr_clf.predict(X_test)

# accuracy와 roc_auc 측정
print('accuracy: {0:.3f}, roc_auc:{1:.3f}'.format(accuracy_score(y_test, lr_preds),
                                                 roc_auc_score(y_test , lr_preds)))
```

In [15]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 로지스틱 회귀를 이용하여 학습 및 예측 수행.
# solver인자값을 생성자로 입력하지 않으면 solver='lbfgs'  
lr_clf = LogisticRegression() # solver='lbfgs'
lr_clf.fit(X_train, y_train)
lr_preds = lr_clf.predict(X_test)

# accuracy와 roc_auc 측정
print('accuracy: {0:.3f}, roc_auc:{1:.3f}'.format(accuracy_score(y_test, lr_preds),
                                                 roc_auc_score(y_test , lr_preds)))

accuracy: 0.723, roc_auc:0.578


C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
lr_clf.predict_proba(X_test)[0] # 정답일 확률/오답일 확률

array([0.34280428, 0.65719572])

#### 더 성능을 높여보자.

재생시간 스케일링

```python
df[['동영상재생시간','실제재생시간']]
```

In [ ]:
# 동영상에서 재생바가 위치한 시간  # 동영상을 본 시긴

In [18]:
df[['동영상재생시간','실제재생시간']]

,동영상재생시간,실제재생시간
0,0,8
1,0,8
2,0,8
3,0,8
4,0,8
...,...,...
99995,0,69
99996,193,67
99997,193,67
99998,193,67


```python
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df[['동영상재생시간','실제재생시간']])
```

In [19]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df[['동영상재생시간','실제재생시간']])

```python
df[['동영상재생시간','실제재생시간']] = data_scaled
```

In [20]:
df[['동영상재생시간','실제재생시간']] = data_scaled

```python
X = df.iloc[:,1:].to_numpy()
Y = df['정오답'].to_numpy()
```

In [23]:
X = df.iloc[:,1:].to_numpy()
Y = df['정오답'].to_numpy()

```python
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train , X_test, y_train , y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
```

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train , X_test, y_train , y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

```python
# 로지스틱 회귀를 이용하여 학습 및 예측 수행.
# solver인자값을 생성자로 입력하지 않으면 solver='lbfgs'  
lr_clf = LogisticRegression() # solver='lbfgs'
lr_clf.fit(X_train, y_train)
lr_preds = lr_clf.predict(X_test)

# accuracy와 roc_auc 측정
print('accuracy: {0:.3f}, roc_auc:{1:.3f}'.format(accuracy_score(y_test, lr_preds),
                                                 roc_auc_score(y_test , lr_preds)))
```

In [25]:
# 로지스틱 회귀를 이용하여 학습 및 예측 수행.
# solver인자값을 생성자로 입력하지 않으면 solver='lbfgs'  
lr_clf = LogisticRegression() # solver='lbfgs'
lr_clf.fit(X_train, y_train)
lr_preds = lr_clf.predict(X_test)

# accuracy와 roc_auc 측정
print('accuracy: {0:.3f}, roc_auc:{1:.3f}'.format(accuracy_score(y_test, lr_preds),
                                                 roc_auc_score(y_test , lr_preds)))

accuracy: 0.751, roc_auc:0.643


C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


```python
pd.DataFrame(data=[lr_preds, y_test])
```

In [26]:
pd.DataFrame(data=[lr_preds, y_test])
# 0: 예측값
# 1: 실제값

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,18410,18411,18412,18413,18414,18415,18416,18417,18418,18419,18420,18421,18422,18423,18424
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
